In [ ]:
from utils import *
import numpy as np

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

## Unit Test R() and Jacobian

In [ ]:
n_hat = np.array([1,0,0])
theta =  0.1 #np.pi/6 #rad

rot_mat_simp = R_simp(n_hat, theta)
print(rot_mat_simp)
angs = np.array([theta,0 ,0])
rot_mat = R(angs)
print(rot_mat)

print(R2Euler(rot_mat))

p_point = np.array([1,1,1]).T

J = jacobian(angs, p_point)

d_rot_mat_simp = dR_simp(n_hat,theta)
# print(d_rot_mat_simp)

## Inline pointcloud rendering using Vedo

In [1]:
## from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget

try:
    plt1.closeWindow()
    print("closed")
except:
    pass

location = 'C:/Users/Derm/2021-03-10-16-43-50_Velodyne-VLP-16-Data_garminSignage.txt'
cloud = np.loadtxt(open(location, "rb"), delimiter=",")
#remove all rows with NaN elements
cloud = cloud[~np.isnan(cloud).any(axis=1)]
plt1 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)
settings.embedWindow(backend='ipyvtk', verbose = True) #was this
# settings.useParallelProjection = True #makes view orthographic
# plt1.show(c, "Volpe Data Test", at = 0)#.close() #WAS THIS

# for debug
# cloud = np.random.randn(10000,3)
# cloud[:,0] += -50*cloud[:,1] + np.random.randn()*5
# cloud[:,1] += 5
# cloud[:,1] = cloud[:,1] * 3 + 10*np.random.randn() + np.sin(cloud[:,1]*5)*10
# cloud[:,2] += 0.5*cloud[:,1] - 10

f =np.array([100,100,2]) #fidelity in x, y, z
# f =np.array([1,1,1]) #fidelity in x, y, z
lim = np.array([-100,100,-100,100,-10,10])
# lim = np.array([-50,50,-50,50,-10,10])

E = subdivide_scan(cloud,plt1, bounds = lim, fid = f, draw_grid = False) 
ViewInteractiveWidget(plt1.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)


## Inline volumetric rendering using ipyvolume

In [ ]:
#simple demo
import ipyvolume
ds = ipyvolume.datasets.aquariusA2.fetch()
short = ds.data[:,:,:]
ipyvolume.quickvolshow(short, lighting=True)

In [ ]:
ans = np.linspace(1,12,8)
ans[2] = 0
print(ans)

test = ans[ans < 10]
print(test)
np.shape(test)[0]

print(np.median(test[:3]))

In [ ]:
print(E)